In [1]:
import sklearn 
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import pandas as pd 
import numpy as np
from statsmodels.tsa.arima_model import ARMA
import datetime
from sklearn.model_selection import train_test_split

/home/cody/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
full_df = pd.read_pickle("../ncaa_data/full_df.p")

# Filter Down to Rows Where Each Team Has Actually Played a Previous Game So We Have Data

In [3]:
populated_df = full_df[(full_df["FGM"] > 0) & (full_df["FGM2"] > 0)]

In [4]:
y = populated_df["Team1Score"] - populated_df["Team2Score"]

In [5]:
exclude = ["Team1Score","Team2Score", "date","DayZero","Score","DayNum","Season","WScore","LScore","Team1","Team2","DayZero2","Score2","DayNum2","Season2","date2"]
feature_columns = populated_df.columns
feature_columns = [item for item in feature_columns if item not in exclude]

# Split It Up and Train a Model

In [6]:
x_train, x_test, y_train, y_test = train_test_split(populated_df[feature_columns], y, test_size=0.1)

In [7]:
linear_model = LinearRegression()
linear_model.fit(x_train.fillna(0), y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [8]:
predicted = linear_model.predict(x_test.fillna(0))

In [9]:
len(feature_columns)

513

In [10]:
len(linear_model.coef_)

513

In [11]:
[print (item) for item in list(zip(feature_columns,linear_model.coef_))]

('FGM', 0.087217294427754277)
('FGA', -0.085379837773031486)
('FGM3', -0.01813956048678202)
('FGA3', 0.012632479595875729)
('FTM', 0.042415649560455071)
('FTA', -0.036345382005178145)
('OR', 0.13418444825127812)
('DR', 0.02566663420707465)
('Ast', 0.038679604466688139)
('TO', -0.064436714748446422)
('Stl', 0.053076294755364864)
('Blk', 0.037665789915236385)
('PF', 0.014574646194586009)
('7OT', 0.15039527109637468)
('ACU', -0.067648148307868861)
('ADE', -0.40379930440506318)
('AP', 0.072540507850815103)
('ARG', -0.38710320203748405)
('AUS', -0.031342497151601735)
('BBT', -0.066550138505919468)
('BCM', -0.028537213794136821)
('BD', -0.04331398274824453)
('BIH', -0.052362010076781706)
('BKM', 0.20548994095010756)
('BLS', -0.073773643838513148)
('BNM', -0.060308925072991626)
('BOB', -0.016881630577146145)
('BOW', -0.42496168882812574)
('BP5', 0.059127175731582313)
('BPI', -0.051411006559034557)
('BRZ', -0.089057834146635248)
('BUR', -0.011254739364964721)
('BWE', 0.1544018767869188)
('CJB'

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [12]:
csv_file = open("submission.csv", "w+")
team_ids = list(tourney_matchups["team1"])
team_ids2 = list(tourney_matchups["team2"])
seasons = list(tourney_matchups["Season"])
csv_file.write("ID,Pred\n")
for i in range(len(tourney_matchups["Season"])):
    csv_string = str(seasons[i]) + "_" + str(team_ids[i]) + "_" + str(team_ids2[i])  + "," + str(predicted[i]) + "\n"
    csv_file.write(csv_string)

NameError: name 'tourney_matchups' is not defined